In [12]:
import pandas as pd
import numpy as np
import torch
!pip install chess
import chess
import chess.engine
import pandas as pd

import sys, importlib
sys.path.append(r"C:\Users\Artem\Desktop\vs code project\chess_forecast\src")

importlib.reload(importlib.import_module('move'))
importlib.reload(importlib.import_module('mymodel'))


import move, mymodel

������� ������� ��������: 1251



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
path = '../fens_training_set.csv'
df = pd.read_csv(path)
df.head(5)

,fen,move
0,7k/5p1p/p2p1Pr1/1p4pQ/8/P1P5/2pr3P/2R2K2 w - -...,h5f3
1,5r2/2p3k1/1p3pb1/p1pPr2p/2P1PRP1/3B1R2/PP4K1/8...,g4h5
2,r2r2k1/p3bpp1/q2Bp2p/2pn4/2b1N1Q1/8/PPP2PPP/3R...,d6e7
3,8/8/8/7K/1k6/8/6P1/8 w - - 0 1,g2g4
4,rnbqkb1r/pp3ppp/5n2/3p4/2pP4/3B1N2/PPP2PPP/RNB...,f1e1


In [14]:
len(df)

268550

In [20]:
for i in range(len(df)):
    fen = df.iloc[i, 0]  
    move = df.iloc[i, 1] 
    try:
        board = chess.Board(fen)
    except:
        print(f"невалидный FEN: {fen}")
        continue 
    try:
        try:
            move_obj = chess.Move.from_uci(move) 
        except:
            move_obj = board.parse_san(move)
        if move_obj not in board.legal_moves:
            print(f"некорректный ход1: '{move}' для FEN: {fen},  {i}")
    except:
        print(f"некорректный ход2: '{move}' для FEN: {fen},  {i}")


In [15]:
print(len(df))
print(df.loc[232329])
df = df.drop(232329, axis=0)

268550
fen     1r2qk1r/ppb3pb/2p1P3/4p2p/4n3/1P1N3P/P1P3PB/1R...
move                                                 f1h1
Name: 232329, dtype: object


In [16]:
ALL_MOVES = move.generate_all_possible_moves()
MOVE_TO_INDEX = {m.uci(): i for i, m in enumerate(ALL_MOVES)}
INDEX_TO_MOVE = {i: m for i, m in enumerate(ALL_MOVES)}

print(f"всего {len(ALL_MOVES)}")
print(f"уник {len(MOVE_TO_INDEX)}")


test_moves = ["e2e4", "e7e5", "e1g1", "e7e8q", "a7b8n"]
print("\nПроверка некоторых ходов:")
for move_uci in test_moves:
    if move_uci in MOVE_TO_INDEX:
        print(f"{move_uci} да, есть. (индекс {MOVE_TO_INDEX[move_uci]})")
    else:
        print(f"{move_uci} нет")

всего 4208
уник 4208

Проверка некоторых ходов:
e2e4 да, есть. (индекс 827)
e7e5 да, есть. (индекс 3444)
e1g1 да, есть. (индекс 257)
e7e8q да, есть. (индекс 3471)
a7b8n да, есть. (индекс 3182)


In [19]:
# Тест на стандартной начальной позиции
test_fen = "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
test_move = "e2e4"

try:
    res = mymodel.neiro_input(fen, move_uci, ALL_MOVES, MOVE_TO_INDEX)
    print(f"Работаем ")
    print(f"Ход (индекс): {res['target_move']}")
    print(f"Кол-во легальных ходов : {res['legal_moves_mask'].sum().item()}")
    print(f"Размерность доски (тензора): {res['board'].shape}")
    print(f"Размерность доп признаков: {res['additional'].shape}")
except Exception as e:
    print(f"Ошибка: {e}")

# Тест с превращением пешки
test_fen = "4k3/P7/8/8/8/8/8/4K3 w - - 0 1"
test_move = "a7a8q"

try:
    res = mymodel.neiro_input(test_fen, test_move, ALL_MOVES, MOVE_TO_INDEX)
    print(f"\nПревращение работает ")
    print(f"Ход (индекс) : {res['target_move']}")
except Exception as e:
    print(f"\nОшибка: {e}")

Работаем 
Ход (индекс): 3182
Кол-во легальных ходов : 9
Размерность доски (тензора): torch.Size([12, 8, 8])
Размерность доп признаков: torch.Size([17])

Превращение работает 
Ход (индекс) : 3175
